In [1]:
# imports without torch
import numpy as np
import argparse
import os
import os.path
import visdom
from PIL import Image
from tqdm import tqdm_notebook as tqdm
import json
import sys

##########
# imports torch
import torch
import torchvision
import torchvision.models as models
import torchvision.transforms as transforms
from torch.autograd import Variable
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
##########

# imports models
import model.segnet as segnet
import model.encoder as encoder  # label->word
import model.decoder as decoder  # word->label
import zs_dataset_list as datasets

# imports utility
import make_log as flog

###mapping function and GT_list###

In [2]:
tr_map_te=np.asarray([
26,
4,
4,
4,
4,
4,
4,
4,
4,
3,
3,
3,
3,
3,
3,
2,
2,
2,
2,
2,
2,
2,
2,
2,
2,
1,
1,
1,
1,
1,
1,
1,
1,
0,
0,
0,
0,
0,
0,
0,
0,
0,
0,
10,
10,
10,
10,
10,
10,
10,
10,
9,
9,
9,
9,
9,
9,
9,
9,
9,
9,
8,
8,
8,
8,
8,
8,
8,
8,
8,
8,
7,
7,
7,
7,
7,
7,
6,
6,
6,
6,
6,
6,
5,
5,
5,
5,
5,
5,
5,
5,
19,
19,
15,
17,
17,
15,
20,
16,
25,
22,
23,
23,
19,
19,
14,
20,
20,
19,
20,
12,
20,
16,
22,
22,
22,
22,
22,
15,
11,
18,
18,
20,
15,
12,
12,
13,
17,
15,
20,
19,
25,
20,
15,
13,
12,
19,
16,
25,
12,
19,
15,
25,
12,
12,
16,
12,
11,
12,
13,
17,
19,
18,
12,
11,
20,
14,
17,
12,
13,
20,
13,
15,
16,
20,
17,
19,
19,
15,
18,
24,
24,
24,
24,
24,
24,
24,
11,
11,
21,
21,
13,
26,
])
GT_list = [35, 26, 23, 9, 1, 83, 77, 72, 61, 51, 43, 154, 148,
           149, 105, 123, 112, 127, 152, 167, 109, 179, 116, 102,
           175, 99]

↓ define(1)

In [3]:
class p_args:
    load=False
    config="decoder_batch2"
    test=False
    no_cuda=False

In [4]:
# set config
f_config = open(os.path.join("./config", p_args.config + ".json"), "r")
args = json.load(f_config)
for key, value in args.items():
    if value == "true":
        args[key] = True
    elif value == "false":
        args[key] = False
print(args)

# device settings
p_args.cuda = not p_args.no_cuda and torch.cuda.is_available()
USE_CUDA = p_args.cuda

{'model': False, 'encoder': False, 'decoder': True, 'input_nbr': 100, 'target_nbr': 27, 'semantic_nbr': 0, 'batch_size': 2, 'epochs': 1000, 'lr': 0.0001, 'momentum': 0.1, 'seed': 1, 'lamda': 0, 'gamma': 1, 'input_root': './data/train/input', 'target_root': './data/train/zs_target', 'map_root': './data/train/target', 'filenames': './data/train/zs_names.txt', 'semantic_filename': './v_class/class1.txt', 'project_dir': './model/decoder_batch2', 'model_load_pth': '', 'head_load_pth': './model/decoder_batch2/segnet.pth', 'save_pth': 'segnet.pth', 'output_dir': './data/decoder_batch2', 'n_components': 100, 'PCA': False, 'cos_similarity': False, 'jaccard_similarity': False, 'ZSL': True, 'SSE': False}


↓ define(2)

In [5]:
input_nbr = args["input_nbr"]  # 入力次元数
target_nbr = args["target_nbr"]  # 出力次元数

In [6]:
# set the seed
torch.manual_seed(args["seed"])
if p_args.cuda:
    torch.cuda.manual_seed(args["seed"])

↓ define(3)

In [7]:
head = decoder.ConvNet(input_nbr, target_nbr, args["momentum"])

↓ define(4)

In [8]:
if args["encoder"] is True or args["decoder"] is True:
    if USE_CUDA:  # convert to cuda if needed
        head.cuda()
    else:
        head.float()
    head.eval()
    print(head)

ConvNet(
  (conv1): Conv2d(100, 100, kernel_size=(1, 1), stride=(1, 1))
  (bn1): BatchNorm2d(100, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv2d(100, 27, kernel_size=(1, 1), stride=(1, 1))
)


↓ define(5)

In [9]:
# Create visdom
vis = visdom.Visdom()

# init window
if p_args.test is False:
    """encoder or decoder"""
    win = vis.line(
        X=np.array([0]),
        Y=np.array([0]),
        opts=dict(
            title='train_loss',
            xlabel='epoch',
            ylabel='loss',
            width=800,
            height=400
        )
    )
    win_acc = vis.line(
        X=np.array([0]),
        Y=np.array([0]),
        opts=dict(
            title='train_accuracy',
            xlabel='epoch',
            ylabel='accuracy',
            width=800,
            height=400
        )
    )

In [10]:
# Create log model
f_log = flog.make_log(args["project_dir"])

↓ define(6)

In [11]:
optimizer = optim.Adam(head.parameters(), lr=args["lr"])

# Train Code

In [12]:
def head_train(epoch, trainloader):
    # set head to train mode
    if args["model"] is True:
        model.eval()
        head.train()
    else:
        head.train()

    # define total_loss
    total_loss = 0

    # define epoch_size
    epoch_size = len(trainloader)

    # define batch_loss
    batch_loss = 0

    # define lamda,gamma
    lamda = args["lamda"]
    gamma = args["gamma"]
        
    # define a loss
    if USE_CUDA:
        if args["decoder"] is True:
            """decoder"""
            loss = nn.CrossEntropyLoss(size_average=True).cuda()
        else:
            sys.exit("model is not defined")
        l1_loss = nn.L1Loss(size_average=False).cuda()
    else:
        if args["decoder"] is True:
            """decoder"""
            loss = nn.CrossEntropyLoss(size_average=True)
        else:
            sys.exit("model is not defined")
        l1_loss = nn.L1Loss(size_average=False)
    
    # define tqdm message
    t = tqdm(trainloader, desc="loss", leave=False)

    # iteration over the batches
    for batch_id, data in enumerate(t):
        # make batch tensor and target tensor
        if args["model"] is True and args["decoder"] is True:
            input = data["input"]
            target = data["target"]
        elif args["decoder"] is True:
            input = data["input"]
            target = data["target"]
        else:
            sys.exit("model is not defined")

        if USE_CUDA:
            if args["model"] is True and args["decoder"] is True:
                input = input.cuda()
                target = target.cuda()
            elif args["decoder"] is True:
                input = input.cuda()
                target = target.cuda()
            else:
                sys.exit("model is not defined")

        # initialize gradients
        optimizer.zero_grad()

        # predictions
        if args["model"] is True and args["decoder"] is True:
            semantic = model(input)
            output = head(semantic)
        elif args["decoder"] is True:
            output = head(input)
        else:
            sys.exit("model is not defined")

        # calculate loss
        l_ = loss(output, target)
        if lamda != 0:
            reg_loss = 0
            for param in head.parameters():
                if USE_CUDA:
                    param_target = torch.zeros(param.size()).cuda()
                else:
                    param_target = torch.zeros(param.size())
                reg_loss += l1_loss(param, param_target)

            reg_loss = lamda * reg_loss
            l_ = l_ + reg_loss

        total_loss += l_.item()
        # backward loss
        l_.backward()
        # optimizer step
        optimizer.step()

        # train conditions
        if lamda != 0:
            t.set_description("reg_loss=%f, loss=%f" % (reg_loss.item(), l_.item()))
        else:
            t.set_description("loss=%f" % (l_.item()))

        # visualize train conditions
        if batch_id % 30 == 0 and batch_id != 0:
            batch_loss = batch_loss + l_.item()
            batch_loss = batch_loss / 30
            # display visdom board
            phase = epoch + batch_id / epoch_size
            visualize(phase, batch_loss, win)
            batch_loss = 0
            # evaluate
            """decoder only"""
            head.eval()
            if args["model"] is True and args["decoder"] is True:
                target = data["map"]
                output = head(semantic)
            elif args["decoder"] is True:
                output = head(input)
            else:
                sys.exit("model is not defined")
            head_evaluate(output, target, epoch, epoch_size, batch_id)
            head.train()
        else:
            batch_loss = batch_loss + l_.item()

    return total_loss

# Test Code

In [13]:
def head_test(testloader):
    # set model to eval mode
    head.eval()

    # make output_dir
    if not os.path.isdir(args["output_dir"]):
        os.makedirs(args["output_dir"])

    # iteration over the batches
    for batch_id, data in enumerate(tqdm(testloader)):
        # make batch tensor and target tensor
        input = data['input']

        if USE_CUDA:
            input = input.cuda()

        # predictions
        output = model(input)

        # output segmentation_img
        if batch_id < 10:
            filename = "00"+batch_id
        elif batch_id < 100:
            filename = "0"+batch_id
        else:
            filename = batch_id
        filename = filename + ".png"
        single_output = output[0, :, :, :]
        te_result = single_output.max(0)[1].cpu().numpy()
        result = np.uint8(te_result)

        Image.fromarray(result).save(
            os.path.join(args["output_dir"], filename))

# Eval Code

In [14]:
def head_evaluate(output, target_map, epoch, epoch_size, batch_id):
    accuracy_te=[]
    for id in range(output.size(0)):
        single_output = output[id, :, :, :]
        target = target_map[id, :, :].cpu().numpy()
        te_result = single_output.max(0)[1].cpu().numpy()
        result = te_result == target
        if len(result) != 0:
            acc_te = result.mean()
            accuracy_te.append(acc_te)

    acc_te = np.asarray(accuracy_te)
    acc = np.mean(acc_te[~np.isnan(acc_te)])
    phase = epoch + batch_id / epoch_size
    visualize(phase, acc, win_acc)

# Visualizer

In [15]:
def visualize(phase, visualized_data, window):
    vis.line(
        X=np.array([phase]),
        Y=np.array([visualized_data]),
        update='append',
        win=window
    )

# Utility

In [16]:
def min_euclidean(out, sem):
    """pytorch calculate euclidean"""
    nbr = sem.size(1)
    ab = torch.mm(out.view(-1, nbr), sem.t())
    ab = ab.view(out.size(0), out.size(1), sem.size(0))
    aa = (sem**2).sum(1)
    bb = (out**2).sum(-1)
    res = aa[None, None, :] + bb[:, :, None] - 2 * ab
    return res.min(-1)[1]

# Main

In [17]:
# compose transforms
train_transform = transforms.Compose(
    [transforms.RandomResizedCrop(256, scale=(1.0, 1.0),ratio=(1.0, 1.0),
                                  interpolation=Image.NEAREST)]
)
test_transform = transforms.Compose(
    []
)

# initialize model
# model.initialized_with_pretrained_weights()

# load model
if p_args.load is True:
    if args["decoder"] is True:
        head.load_from_filename(args["head_load_pth"])

# load dataset
trainset = datasets.ImageFolderDenseFileLists(
    input_root=args["input_root"], target_root=args["target_root"],
    map_root=args["map_root"], filenames=args["filenames"],
    semantic_filename=args["semantic_filename"], training=True,
    model=None, config=args, transform=train_transform,
    USE_CUDA=USE_CUDA)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=args["batch_size"], shuffle=True,
    num_workers=args["batch_size"])
testset = datasets.ImageFolderDenseFileLists(
    input_root='./data/test/input', target_root=None,
    map_root=None, filenames='./data/test/names.txt',
    semantic_filename=args["semantic_filename"], training=False,
    model=None, config=args, transform=test_transform,
    USE_CUDA=USE_CUDA)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=1, shuffle=False, num_workers=1)

# train and test
for epoch in range(0, args["epochs"]):

    if p_args.test is False:
        # training
        loss = head_train(epoch, trainloader)
        print("epoch:%d,head_loss:%f" % (epoch,loss))
        # open log_file
        f_log.open()
        # write log_file
        f_log.write(epoch, loss)
        # close log_file
        f_log.close()
        # make project_dir
        if not os.path.isdir(args["project_dir"]):
            os.makedirs(args["project_dir"])
        # save checkpoint
        torch.save(head.state_dict(),
                   args["project_dir"] + "/checkpoint_" + str(epoch) +
                   ".pth")
    elif p_args.test is True and p_args.load is True:
        # test
        head_test(testloader)
        break
    else:
        print('can not test the model!')
        break
# save model
if p_args.test is False:
    torch.save(head.state_dict(), os.path.join(
        args["project_dir"], args["save_pth"]))

epoch:0,head_loss:263.950943


epoch:1,head_loss:260.392505


epoch:2,head_loss:258.815088


epoch:3,head_loss:256.646754


epoch:4,head_loss:253.331083


...

epoch:995,head_loss:16.223626


epoch:996,head_loss:12.317070


epoch:997,head_loss:26.702191


epoch:998,head_loss:27.250740


epoch:999,head_loss:15.557569
